In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
random.seed(138)
import sklearn

# Part 1: Assembling a dataframe

commands from *pandas* that might be useful for any ML problem ( particularly your assignment). We will approach this by assembling a dataframe from 3 different data sources. These three files are *df_vlinder.csv*, *df_meta.csv* and *df_synop.csv*. Your final result must be the same as the *df_merged.csv* file. You can use this file as a reference point for your answer for part 1.

Here you will find a list of commands that you might find useful for this notebook:

*   pandas.to_datetime
*   pandas.DataFrame.pop
*   pandas.DataFrame.reset_index
*   pandas.DataFrame.set_index
*   pandas.DataFrame.resample
*   pandas.DataFrame.merge 
*   pandas.to_numeric 
*   pandas.DataFrame.rename 


In [2]:
df_vlinder = pd.read_csv('df_vlinder.csv')

In [3]:
df_vlinder.to_csv('df_vlinder.csv', index=False)

In [4]:
df_vlinder=df_vlinder.rename(columns={"Temperatuur":"TEMP","Neerslagintensiteit": "PRECIP_QUANTITY", "Luchtdruk":"PRESSURE", "Windrichting": "WIND_DIRECTION",
                                    "Windsnelheid": "WIND_SPEED", "Vochtigheid":"HUMIDITY_RELATIVE" })

In [ ]:
df_vlinder.TEMP.plot()

This figure shows the temperature at station vlinder 16 for June and July. You can find this station on the Vlinder dashboard (https://vlinder.ugent.be/dashboard/?stations=vlinder16) Do we want to keep all this data? If not, which month should we perhaps throw out?

Further, we have the date (Datum), and we have the time (Tijd (UTC)) in hours, minutes, and seconds. See if you can combine these into a DateTime column. This will be useful for later on. All the variables left in Dutch will not be used further, so you should delete these columns. As we'll see, when loading in the synop data, there is a mismatch between the frequency at which the measurements are made. How can we solve this? Which data set would we modify for this (vlinder or synop)? Keep in mind that certain variables behave differently when considering different measuring frequencies.


In this following part we load in the metadata. This dataframe contains information of the surrounding area of the station. We want from to add information from this file to be added to our main merged file. Specifically, the altitude (height), longitude (lon), lattitude (lat) and the three land cover fractions (water, green & impervious). These three fractions we specifically want for a buffer size of 150 m (tip look at the values of radius).    

In [6]:
df_meta = pd.read_csv('df_meta.csv' )

In [ ]:
df_meta.head()

In this part, we will load in the data from 2 synoptic stations: one in Antwerpen (Deurne) and one in Stabroek. These two stations can be found on the following map (https://www.google.com/maps/d/u/0/viewer?mid=1tgyYVHdsD4xL_FixPol8NwBc8GY&ll=50.57784013934338%2C4.264776500000016&z=6) These two stations are the closest to vlinder 16. We will use the temperature, wind speed, and radiation information of one of these two stations and append this to our big dataframe. Which one would you choose and why?

In [8]:
df_synop=pd.read_csv('df_synop.csv')
df_synop['datetime']= pd.to_datetime(df_synop['datetime'])
df_synop=df_synop.rename(columns={"temp":"TEMP_SYNOP","wind_speed":"WIND_SPEED_SYNOP","short_wave_from_sky_1hour":"RAD_SYNOP"})

# Part 2 Feeding the data to the model

In [17]:
df_merged=pd.read_csv('df_merged.csv')

This is how your final data frame should look like.

In [ ]:
df_merged.head()

What does this block of code mean?

Why do we have to do this?

In [19]:
df_merged=df_merged.dropna()
df_merged=df_merged.reset_index(drop=True)

We will not need these columns anymore. 

Why is it important to remove these from the data set?

In [ ]:
df_merged.pop('Vlinder')
df_merged.pop('community_name')
df_merged.pop('station')

Calculate here a correlation matrix for the fetaures. What can we learn from this?

This particular choice of test set was made in consideration of making the graph at the bottom the of this notebook. Do you perhaps see any problems with this aproach?

In [21]:
df_test= df_merged.loc[range(15,60)]
df_train= df_merged.loc[range(60,len(df_merged))]

Is this block of code optimal. If not, what could be improved here?

In [22]:
df_test=df_test.reset_index(drop=True)
df_test['DATE']=df_test.TEMP*0
df_test['HOUR']=df_test.TEMP*0
str_date ='' 
str_hour=''  
for i in range(len(df_test)):
  str_tot=df_test.loc[i,'datetime']
  str_date=str_tot[8]+str_tot[9]
  str_hour=str_tot[11]+str_tot[12]
  df_test.loc[i,'DATE']=float(str_date)
  df_test.loc[i,'HOUR']=float(str_hour)

date=df_test['DATE']
hour=df_test['HOUR']
df_test.pop('DATE')
df_test.pop('HOUR')
print('')

In [23]:
df_train.pop('datetime')
df_test.pop('datetime')
df_merged.pop('datetime')
print('')

In [ ]:
df_train.head()

Write here a block of code to scale your data. 

Look at the sklearn preprocessing library for which scaler (I recommend looking into the MinMaxScaler) you can use and what functionalities there are. 
 

In [ ]:
X_train=df_train_scale.drop('TEMP', axis=1)
Y_train=df_train_scale['TEMP']
X_test=df_test.drop('TEMP', axis=1)
Y_test=df_test['TEMP']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val =train_test_split(X_train,Y_train,test_size=0.1)

The ML model of choice here is the K-nearest neighbours regression model. This is a non-parametric model where the output for an input vector is calculated by taking an average of the k-nearest neighbours of that input vector in the feature space. For a more thourough explanation on this look at the documentation: https://scikit-learn.org/stable/modules/neighbors.html

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import math

In this part we will evaluate the model for 20 different values of k on the validation set. With this we will determine which value would be optimal. (Hyperparameter tuning) 

In [ ]:
valRMSE=[]
for i in range(1,20):
    parameter = i
    model = KNeighborsRegressor(parameter)
    model.fit(X_train,Y_train)
    predval = model.predict(X_val)
    valerror = Y_val.values-predval.reshape(len(predval))
    valRMSE.append(np.sqrt(np.mean(np.square(valerror))))

plt.plot(range(1,20), valRMSE,'-*')
plt.xlabel("number of neighbours k")
plt.ylabel("RMSE on the validation set")
plt.title('hyperparameter tuning for KNN')
plt.grid()
#Choose the best parameter value
minindex = np.argmin(valRMSE)
print('Chosen parameter: ' + str(minindex+1))

Now that we have chosen the parameter, we train the model using this optimized value for k. 

In [ ]:
model = KNeighborsRegressor(minindex+1)
model.fit(X_train,Y_train)

Write in this following block of code you should let your model predict for the test set. *pred_T* should be your answer for the model prediction and *actual_T* should be your actual measurments. Think well about in which format your model will return the prediction and what the inverse scaling function requires.

In [ ]:
time=date+hour/24.0
fig = plt.figure(figsize=(10, 4))
plt.plot(time,pred_T,'b')
plt.plot(time,actual_T,'k')
plt.grid()
plt.legend(['Prediction','Actual value'],loc=2)
plt.xlabel("Time (days)")
plt.ylabel("Temperature (°C)")
plt.title('Model performance')

Final part:
Calculate the mean absolute error (MAE), root mean squared error (RMSE) and bias.


Do these values match the figure above? What could we do to improve these values and our prediction in general?